In [19]:
import pandas as pd
import random

# Defining 5 fraud/AML cases
case_alerts_data = [
    {"Case ID": "C001", "Alert Type": "Large Transactions", "Risk Score": 85, "Status": "Open", "Case Created Date": "2025-02-01"},
    {"Case ID": "C002", "Alert Type": "Rapid Movement of Funds", "Risk Score": 92, "Status": "Under Review", "Case Created Date": "2025-02-03"},
    {"Case ID": "C003", "Alert Type": "Pattern in Funds Transfers", "Risk Score": 78, "Status": "Closed", "Case Created Date": "2025-02-05"},
    {"Case ID": "C004", "Alert Type": "Round Dollar Values", "Risk Score": 81, "Status": "Open", "Case Created Date": "2025-02-07"},
    {"Case ID": "C005", "Alert Type": "Large Transactions", "Risk Score": 88, "Status": "Under Review", "Case Created Date": "2025-02-10"},
]

# Converting to DataFrame
case_alerts_df = pd.DataFrame(case_alerts_data)

# Generating transactions dataset
transactions_data = []
for case in case_alerts_data:
    num_transactions = random.randint(1, 20)  # 1-20 transactions per case
    case_id = case["Case ID"]
    for i in range(num_transactions):
        transactions_data.append({
            "Transaction ID": f"T{random.randint(1000, 9999)}",
            "Case ID": case_id,
            "Amount": round(random.uniform(1000, 50000), 2),  # Random transaction amount
            "Transaction Date": f"2025-02-{random.randint(1, 15)}",
            "Account Number": f"ACC{random.randint(10000, 99999)}"
        })

# Converting to DataFrame
transactions_df = pd.DataFrame(transactions_data)

# Displayin the first few rows
print(case_alerts_df)
print(transactions_df.head())

  Case ID                  Alert Type  Risk Score        Status  \
0    C001          Large Transactions          85          Open   
1    C002     Rapid Movement of Funds          92  Under Review   
2    C003  Pattern in Funds Transfers          78        Closed   
3    C004         Round Dollar Values          81          Open   
4    C005          Large Transactions          88  Under Review   

  Case Created Date  
0        2025-02-01  
1        2025-02-03  
2        2025-02-05  
3        2025-02-07  
4        2025-02-10  
  Transaction ID Case ID    Amount Transaction Date Account Number
0          T4487    C001  43790.41        2025-02-1       ACC56692
1          T9719    C001  39217.96        2025-02-9       ACC95963
2          T4197    C001  33125.56       2025-02-15       ACC79613
3          T5536    C001   5155.13        2025-02-5       ACC15802
4          T7884    C001  27363.43       2025-02-10       ACC88539


In [21]:
# Save the case_alerts DataFrame to a CSV file
case_alerts_df.to_csv('case_alerts.csv', index=False)

# Save the transactions DataFrame to a CSV file
transactions_df.to_csv('transactions.csv', index=False)

In [37]:
print(case_alerts_df.dtypes)
print(transactions_df.dtypes)

Case ID              object
Alert Type           object
Risk Score            int64
Status               object
Case Created Date    object
dtype: object
Transaction ID       object
Case ID              object
Amount              float64
Transaction Date     object
Account Number       object
dtype: object


In [39]:
print(case_alerts_df.isnull().sum())
print(transactions_df.isnull().sum())

Case ID              0
Alert Type           0
Risk Score           0
Status               0
Case Created Date    0
dtype: int64
Transaction ID      0
Case ID             0
Amount              0
Transaction Date    0
Account Number      0
dtype: int64


In [41]:
print("Duplicate Transaction IDs:", transactions_df["Transaction ID"].duplicated().sum())
print("Missing Case IDs in transactions:", transactions_df[~transactions_df["Case ID"].isin(case_alerts_df["Case ID"])].shape[0])

Duplicate Transaction IDs: 0
Missing Case IDs in transactions: 0


In [43]:
total_amounts = transactions_df.groupby("Case ID")["Amount"].sum().reset_index()

In [45]:
avg_risk_score = case_alerts_df.groupby("Alert Type")["Risk Score"].mean().reset_index()

In [47]:
print(total_amounts)
print(avg_risk_score)

  Case ID     Amount
0    C001  192637.57
1    C002  505814.58
2    C003   77338.07
3    C004  249669.00
4    C005  156936.25
                   Alert Type  Risk Score
0          Large Transactions        86.5
1  Pattern in Funds Transfers        78.0
2     Rapid Movement of Funds        92.0
3         Round Dollar Values        81.0
